# ASTR-80 Testing: U-Net Inference + Persistence + MLflow
This notebook validates the end-to-end path: candidates (ASTR-79) -> UNet inference -> detections persist -> optional MLflow logging.


In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))
import numpy as np
from uuid import uuid4
from sqlalchemy.ext.asyncio import AsyncSession
from src.core.db.session import AsyncSessionLocal
from src.domains.detection.services.model_inference import ModelInferenceService
from src.domains.detection.config import ModelConfig
print('✅ Imports OK')


In [ ]:
# Synthetic difference image and candidates
H, W = 512, 512
D = np.zeros((H, W), dtype=np.float32)
candidates = [{"pixel_x": 256, "pixel_y": 256}]
observation_id = str(uuid4())
model_run_id = str(uuid4())
print('Obs:', observation_id, 'Run:', model_run_id)


In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"  # not 0.0.0.0
os.environ["MLFLOW_TRACING_ENABLED"] = "false"
print("MLFLOW_TRACKING_URI=", os.environ["MLFLOW_TRACKING_URI"])

In [ ]:
import mlflow, requests
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])  # reuse env
# Quick health check
try:
    r = requests.get(os.environ["MLFLOW_TRACKING_URI"].rstrip("/") + "/health", timeout=5)
    print("MLflow /health:", r.status_code)
except Exception as e:
    print("MLflow health check failed:", e)

with mlflow.start_run() as run:
    mlflow.log_param("ping", "ok")
    print("run_id:", run.info.run_id)

In [ ]:
import asyncio, mlflow
async def run_infer():
    async with AsyncSessionLocal() as db:  # type: ignore
        svc = ModelInferenceService(ModelConfig())
        return await svc.infer_and_persist_candidates(db, observation_id, D, candidates, model_run_id=model_run_id)

# Run and print run link if possible
res = await run_infer()
print(res)
# We started our own run earlier (ping cell). No active run may be present here.

